In [1]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2023-01-10 20:52:27--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.199.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   233MB/s    in 0.3s    

2023-01-10 20:52:27 (233 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



# Preprocessing

In [3]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.9.2'

In [4]:
dataset_path = './cats_and_dogs_filtered.zip'

In [5]:
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')

In [6]:
zip_object.extractall('./')

In [7]:
zip_object.close()

In [8]:
dataset_path_new = './cats_and_dogs_filtered'

In [9]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

# Build model

In [10]:
img_shape = (128, 128, 3)

In [23]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape,
                                               include_top = False, # el encabezado lo pondremos nosotros
                                               weights = "imagenet")

In [24]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

Congelamos el modelo base

In [13]:
base_model.trainable = False

In [14]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [16]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [17]:
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(global_average_layer)

In [18]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [22]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate = 0.0001),
              loss = "binary_crossentropy", metrics =["accuracy"])